- Ressources de HLS absente de dbpedia

In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
env = 'staging'
pk_project = 153
debug = True
execute = False

import pandas as pd
import duckdb
import threading
from icecream import ic

import geovpylib.utils as u
import geovpylib.database as db
import geovpylib.pks as pks
import geovpylib.sparql as sparql
import geovpylib.graphs as graphs
import geovpylib.find as find

eta = u.Eta()
ic.configureOutput(prefix='debug| ')
if debug: ic.enable()
else: ic.disable()


db.connect(env, pk_project, execute=execute)

Requests will not be executed
=== Setting STAGING environment ===
>> Connecting to PGSQL Database ... Connected!


## HLS humans found in wikidata

### Fetch data

In [27]:
sparql.init('https://query.wikidata.org/sparql')
wikidata = sparql.query("""
    SELECT ?uri_hls ?uri_wikidataLabel ?uri_wikidata ?genderLabel ?birthdate ?birthplace ?deathdate ?deathplace ?description
    WHERE {
        ?uri_wikidata wdt:P902 ?uri_hls .
        ?uri_wikidata wdt:P31 wd:Q5 .
        optional { ?uri_wikidata wdt:P21 ?gender . }
        optional { ?uri_wikidata wdt:P569 ?birthdate . }
        optional { ?uri_wikidata wdt:P19 ?birthplace . }
        optional { ?uri_wikidata wdt:P570 ?deathdate . }
        optional { ?uri_wikidata wdt:P20 ?deathplace . }
        SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }

        SERVICE wikibase:label { 
            bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" . 
            ?uri_wikidata schema:description ?description .
        }
    }
""")
                        
wikidata.rename(columns={'uri_wikidataLabel': 'name', 'genderLabel': 'gender'}, inplace=True)
wikidata['uri_hls'] = 'https://hls-dhs-dss.ch/articles/' + wikidata['uri_hls']

wikidata = wikidata[['name', 'description', 'uri_wikidata', 'uri_hls', 'gender', 'birthdate', 'birthplace', 'deathdate', 'deathplace']]
wikidata['gender'] = wikidata['gender'].replace('male', 'Male')
wikidata['gender'] = wikidata['gender'].replace('female', 'Female')
wikidata['birthdate'] = [u.parse_date(strdate) for strdate in wikidata['birthdate']]
wikidata['deathdate'] = [u.parse_date(strdate) for strdate in wikidata['deathdate']]

u.infos(wikidata, random=True)

Shape:  (25354, 9) - extract:


,name,description,uri_wikidata,uri_hls,gender,birthdate,birthplace,deathdate,deathplace
24837,Johannes Walser (1773),Swisse merchand (1773–1833),http://www.wikidata.org/entity/Q91230637,https://hls-dhs-dss.ch/articles/019320,Male,"(1773, 1, 2)",http://www.wikidata.org/entity/Q68315,"(1833, 5, 5)",http://www.wikidata.org/entity/Q13666
10460,Camille Guggenheim,(1894-1930),http://www.wikidata.org/entity/Q78063482,https://hls-dhs-dss.ch/articles/043800,Male,"(1894, 2, 9)",NaN,"(1930, 9, 0)",NaN
18674,Hans Schriber,Swiss politician and historian,http://www.wikidata.org/entity/Q2712188,https://hls-dhs-dss.ch/articles/021241,Male,"(1500, 1, 1)",NaN,<NA>,NaN
20070,Moritz Gartenhauser,NaN,http://www.wikidata.org/entity/Q94489282,https://hls-dhs-dss.ch/articles/019339,Male,"(1450, 1, 1)",NaN,"(1550, 1, 1)",NaN
1420,Charles Didier,Swiss writer and poet (1805-1864),http://www.wikidata.org/entity/Q2958916,https://hls-dhs-dss.ch/articles/015938,Male,"(1805, 9, 5)",http://www.wikidata.org/entity/Q71,"(1864, 3, 7)",http://www.wikidata.org/entity/Q90


In [28]:

sparql.init('https://dbpedia.org/sparql')
count = sparql.query("""
    SELECT 
        count(*) as ?count
    WHERE {
        ?dbpediaURI owl:sameAs ?wikidataURI .
    }
""").iloc[0]['count']
count = int(count)

eta.begin(count, "Fetching all wikidata URI from dbpedia")
offset = 0
uris = []
while offset < count:
    query_result = sparql.query("""
        SELECT 
            ?dbpediaURI ?wikidataURI 
        WHERE {
            ?dbpediaURI owl:sameAs ?wikidataURI .
            FILTER (CONTAINS(STR(?wikidataURI), 'wikidata'))
        }
        LIMIT 10000  OFFSET """ + str(offset) + """
    """)
    if len(uris) != 0: uris.append(query_result)
    eta.print(f'{len(query_result)} added')

    offset += 10000
    eta.iter(offset)
eta.end()

dbpedia = pd.concat(uris)

156 added                                                                              
0 added                                                                                


KeyboardInterrupt: 

In [29]:
# Get URI from dbpedia
sparql.init('https://dbpedia.org/sparql')
dbpedia = sparql.query("""
    SELECT 
        ?dbpediaURI ?wikidataURI # count(*) as ?count #
    WHERE {
        ?dbpediaURI owl:sameAs ?wikidataURI .
        # FILTER (CONTAINS(STR(?wikidataURI), 'wikidata'))
    }
    #LIMIT 100000
""")

u.infos(dbpedia)

Shape:  (10000, 2) - extract:


,dbpediaURI,wikidataURI
0,http://www.openlinksw.com/schemas/oplweb#Busin...,http://www.openlinksw.com/dataspace/organizati...
1,http://www.openlinksw.com/schemas/oplweb#Busin...,http://dbpedia.org/resource/OpenLink_Software
2,"http://dbpedia.org/resource/Bonham,_Texas",http://rdf.freebase.com/ns/m.0100_b
3,http://dbpedia.org/resource/Sthenognatha_cinda,http://rdf.freebase.com/ns/m.0100b5jm
4,http://dbpedia.org/resource/Symphlebia_aryllis,http://rdf.freebase.com/ns/m.0100b6gn


In [30]:
dbpedia[dbpedia['entity'] == 'http://dbpedia.org/resource/Saint-Félix']

KeyError: 'entity'

### Find existing persons in Geovistory

In [31]:
display(find.find_persons_by_uri(wikidata['uri_wikidata']))
display(find.find_persons_by_uri(wikidata['uri_hls']))

,uri,pk_entity


,uri,pk_entity


In [32]:
similars = find.find_persons(wikidata, index_col_name='uri_hls', jobs=10)
u.infos(similars)

wikidata = wikidata.merge(similars[['uri_hls', 'pk_entity']].drop_duplicates(), on='uri_hls', how='left').rename(columns={'pk_entity':'pk_person'})
wikidata['pk_person'] = wikidata['pk_person'].astype(pd.Int64Dtype())
u.infos(wikidata)

Checking data integrity... Done
Find all persons in Geovistory... 153963 found.
Finding similar persons is done - Elapsed: [00h03'17]                   
Shape:  (23, 10) - extract:


,uri_hls,pk_entity,new_name,gv_name,new_gender,gv_gender,new_birthdate,gv_birthdate,new_deathdate,gv_deathdate
9,https://hls-dhs-dss.ch/articles/009482,26225,Grock,grock,Male,NaN,"(1879, 12, 31)","(1880, 1, 10)","(1959, 7, 4)",<NA>
11,https://hls-dhs-dss.ch/articles/012480,1100928,Anna,anna,Female,Female,<NA>,<NA>,<NA>,<NA>
12,https://hls-dhs-dss.ch/articles/012480,1788474,Anna,anna,Female,Female,<NA>,"(1734, 10, 24)",<NA>,"(1782, 6, 13)"
20,https://hls-dhs-dss.ch/articles/012518,759243,Boso,goso,NaN,Male,<NA>,"(1550, 1, 11)",<NA>,<NA>
21,https://hls-dhs-dss.ch/articles/012518,6175771,Boso,boso,NaN,Male,<NA>,"(1574, 1, 11)",<NA>,<NA>


Shape:  (25363, 10) - extract:


,name,description,uri_wikidata,uri_hls,gender,birthdate,birthplace,deathdate,deathplace,pk_person
0,Anatoly Lunacharsky,Russian Marxist revolutionary (1875-1933),http://www.wikidata.org/entity/Q18809,https://hls-dhs-dss.ch/articles/028377,Male,"(1875, 1, 3)",http://www.wikidata.org/entity/Q156747,"(1933, 2, 6)",http://www.wikidata.org/entity/Q180083,<NA>
1,Louis de Rougemont,explorer with false claims,http://www.wikidata.org/entity/Q20014,https://hls-dhs-dss.ch/articles/044295,Male,"(1847, 1, 2)",http://www.wikidata.org/entity/Q678839,"(1921, 6, 9)",NaN,<NA>
2,Élisée Reclus,French geographer and writer,http://www.wikidata.org/entity/Q20951,https://hls-dhs-dss.ch/articles/028424,Male,"(1830, 3, 5)",http://www.wikidata.org/entity/Q320233,"(1905, 7, 4)",http://www.wikidata.org/entity/Q270633,<NA>
3,Hermann Hesse,German writer (1877–1962),http://www.wikidata.org/entity/Q25973,https://hls-dhs-dss.ch/articles/011946,Male,"(1877, 7, 2)",http://www.wikidata.org/entity/Q487921,"(1962, 8, 9)",http://www.wikidata.org/entity/Q694086,<NA>
4,Paul Klee,Swiss artist (1879-1940),http://www.wikidata.org/entity/Q44007,https://hls-dhs-dss.ch/articles/012028,Male,"(1879, 2, 8)",http://www.wikidata.org/entity/Q69071,"(1940, 6, 9)",http://www.wikidata.org/entity/Q69961,<NA>


In [ ]:
to_create = wikidata[pd.isna(wikidata['pk_person'])]
to_update = wikidata[pd.notna(wikidata['pk_person'])]

In [ ]:
to_update

## Create new persons

In [ ]:
# Create persons entities
to_create['pk_person'] = db.resources.create(pks.classes.person, len(to_create))

In [ ]:
# Add their definition
selection = to_create[['pk_person', 'description']].dropna()

graphs.add_definitions(
    selection['pk_person'].tolist(),
    selection['description'].tolist(),
    pks.languages.english
)

In [ ]:
# Add URIs - Wikidata
selection = to_create[['pk_person', 'uri_wikidata']].dropna()

graphs.add_uris(
    selection['pk_person'].tolist(), 
    selection['uri_wikidata'].tolist()
)

In [ ]:
# Add URIs - HLS
selection = to_create[['pk_person', 'uri_hls']].dropna()

# Execute
graphs.add_uris(
    selection['pk_person'].tolist(),
    selection['uri_hls'].tolist()
)

In [ ]:
# Add Gender to person
selection = to_create[['pk_person', 'gender']].dropna()

db.statements.create(
    selection['pk_person'].tolist(),
    pks.properties.person_hasGender_gender,
    selection['gender'].replace('Male', pks.entities.pk_gender_male).replace('Female', pks.entities.pk_gender_female)
)

In [ ]:
# Add birthdate
selection = to_create[['pk_person', 'birthdate']].dropna()
years = [d[0] for d in selection['birthdate']]
months = [d[1] for d in selection['birthdate']]
days = [d[2] for d in selection['birthdate']]

pk_births = db.resources.create(pks.classes.birth, len(selection))
pk_time_prim = db.time_primitives.create(years, months, days, '1 day')

db.statements.create(
    pk_births, 
    pks.properties.birth_broughtIntoLife_person,
    selection['pk_person'].tolist()
)

db.statements.create(
    pk_births, 
    pks.properties.timeSpan_atSomeTimeWithin_timePrimitive,
    pk_time_prim
)

In [ ]:
# Add Deathdate
selection = to_create[['pk_person', 'deathdate']].dropna()
years = [d[0] for d in selection['deathdate']]
months = [d[1] for d in selection['deathdate']]
days = [d[2] for d in selection['deathdate']]

pk_deaths = db.resources.create(pks.classes.death, len(selection))
pk_time_prim = db.time_primitives.create(years, months, days, '1 day')

db.statements.create(
    pk_deaths, 
    pks.properties.death_wasDeathOf_person,
    selection['pk_person'].tolist()
)

db.statements.create(
    pk_deaths, 
    pks.properties.timeSpan_atSomeTimeWithin_timePrimitive,
    pk_time_prim
)